### Attention Mechanisms
In this notebook, we will sequentially implement different variants of attention mechanisms. These variants will build on each other, with the goal of finally creating a compact, efficient implementation of an attention mechanism, which we can then plug into our LLM architecture.

#### Why Attention?
In machine translation, it is not possible to merely translate word by word. The translation process requires contextual understanding and grammatical alignment.

- "Kannst du mir helfen diesen Satz zu uebersetzen" should not be translated to "Can you me help this sentence to translate", but rather to "Can you help me translate this sentence".
- Certain words require access to words appearing before or later in the original sentence. For instance, the verb "to translate" should be used in the context of "this sentence", and not independently.

Typically, to overcome this challenge, deep neural networks with two submodules are used:

- **encoder**: first read in and process the entire text (already done in the `preprocessing.ipynb` notebook).

- **decoder**: produces the translated text.

Pre-LLM architectures typically involved recurrent neural networks, a type of neural network where outputs from previous steps are fed as inputs to the current step, making them well-suited for sequential data. In this many-to-one RNN architecture, the input text is fed token by token into the encoder, which processes it sequentially. The terminal state of the encoder is a memory cell, known as the hidden state, which encodes the entire input. This hidden state is then fed to a decoder that would then generate the translated sentence, word by word, one word at a time.

|     ![RNNEncoder-Decoder](../images/RNNencoderdecoder.png)     |
|:-----------------------------------------------------------:|
| *RNN Encoder Decoder* (Dive Into Deep Learning Chapter 10.7 |

- While the encoder is many-to-one, the decoder is a one-to-many architecture, since the hidden state is passed at every step of the decoding process.
- While it is not strictly necessary to understand the inner workings of encoder-decoder RNNs to develop an LLM, the `seq2seq.ipynb` notebook in stage 1 aims to more deeply explore these architectures.

**encoder-decoder RNNs had many shortcomings that motivated the design of attention mechanisms**, namely that it was not possible to access earlier hidden states from the encoder during the decoding phase, since we rely on a single hidden state containing all the relevant information. Context was lost, especially in complex sentences where dependencies span larger distances.



## Simple Self-Attention

The original *transformer* architecture includes a 'Self-Attention' mechanism inspired by the Badhanau attention mechanism mentioned in `seq2seq.ipynb`.

A mechanism that uses self-attention allows each position in the input sequence (each word), to consider the importance of all other positions (all other words) in the same sequence when creating the embedding of such sequence.

In short, **the goal of a self-attention mechanism is to, for each position in the input sequence, compute a context vector that captures, quantifies, and combines information from all other positions.** For example, given an input vector $X = {x^{(1)}, x^{(2)},..., x^{(T)}}$ (which represents a text that has already been transformed into token embeddings), we want to compute the context vector $z^{(3)}$ of the 3rd position, $x^{(3)}$. The importance of each input for computing $z^{(3)}$ is determined by attention weights ${\alpha_{3,1}, \alpha_{3,2},..., \alpha_{3, T}}$, being $z^{(3)}$ a combination of all input vectors weighted with respect to input element $x^{(3)}$. In math notation:

$$
z^{(3)} = \sum^{T}_{j=1} \alpha_{3, j} \space x^{(j)}
$$

where the attention weights $\alpha_{3, j}$ quantify how much token $x^{(j)}$ should influence token $x^{(j)}$

- Our goal is to compute context vectors $z^{(i)}$ for all $x^{(i)}$ in the input sequence. The resulting context vectors, we can say, are enriched, more informational embedding vectors.
- The collection $\{z^{(1)}, z^{(2)}, z^{(3)}, \dots, z^{(T)}\}$ forms a set of context-aware embeddings that will feed subsequent layers of the model.

#### Why 'Self-Attention'

The 'self' refers to the fact that the mechanism computes weights by assessing and learning the dependencies of different positions **within the same sequence**. The relationships of the various parts of the input itself are considered. This is in contrast to attention mechanisms that focus on assessing and learning the relationship of elements that are part of distinct sequences. Sequence-to-sequence models, for instance, where assessment might be done over an input sequence and a distinct output sequence is not a self-attentive mechanism.




In [1]:
import torch
import numpy as np

inputs = torch.tensor(
    [[0.35, 0.15, 0.89], #These (x^1)
     [0.97, 0.8, 0.3], #are (x^2)
     [0.65, 0.34, 0.24], #random (x^3)
     [0.2, 0.87, 0.34], #words (x^4)
     [0.86, 0.13, 0.05], #and (x^5)
     [0.10, 0.20, 0.30]] #embeddings (x^6)
)

print(f'This is the third token: {inputs[2]}')

print(f'We initialize an empty vector to calculate intermediate attention scores, one for each row (word) of our embeddings: {torch.empty(inputs.shape[0])}')

This is the third token: tensor([0.6500, 0.3400, 0.2400])
We initialize an empty vector to calculate intermediate attention scores, one for each row (word) of our embeddings: tensor([0., 0., 0., 0., 0., 0.])


In [2]:

def calculate_intermediate_attention_score_for_one_position(inputs):
    query = inputs[2] # we are working with the third input token as the query
    attn_scores_3 = torch.empty(inputs.shape[0])
    for i, x_i in enumerate(inputs):
        attn_scores_3[i] = torch.dot(query, x_i)
    return attn_scores_3


attention_scores_3 = calculate_intermediate_attention_score_for_one_position(inputs)
print(f'The intermediate attention scores for the second token are: {attention_scores_3}')

The intermediate attention scores for the second token are: tensor([0.4921, 0.9745, 0.5957, 0.5074, 0.6152, 0.2050])


#### Dot Product Intuition

In the code above, we have calculated intermediate values for the attention weights of the embedded query token $x^{(2)}$. We have simply initialized an empty, 6-dimension vector (one for each embedded input token) and set each value to be the dot product between input $x^{(i)}$ and query $x^{(2)}$.

Recall that, for two vectors $u = [u_{1}, u_{2}, ..., u_{n}]$ and $v = [v_{1}, v_{2}, ..., v_{3}]$ in $R^{n}$, the dot product is defined as the sum of the element-wise multiplication:

$$
\begin{align*}
u \bullet v = \sum_{i=1}^{n} u_{i}v_{i} = u_{1}v_{1} + u_{2}v_{2} + ... + u_{n}v_{n}
\end{align*}
$$

Alternatively, the dot product can also be geometrically expressed as:

$$
\begin{align*}
u \bullet v = |u||v|cos(\theta)
\end{align*}
$$

where $|u|$ and $|v|$ are the magnitudes of the vectors, and $\theta$ is the angle between them.

- When $\theta = 0^\circ$, the vectors are parallel and point in the same direction. $cos(0) = 1$, and therefore $u \bullet v = |u||v|$. This is the maximum possible value of the dot product for given magnitudes.
- When $\theta = 90^\circ$, the vectors are perpendicular. $cos(90) = 0$, and $u \bullet v = 0$. This indicates no alignment or similarity. The vectors share no directional overlap.
- When $\theta = 180^\circ$, the vectors are parallel but point to opposite directions. $cos(180) = -1$ and $u \bullet v = -|u||v|$. This negative value indicates dissimilarity.

**The dot product depends on both alignment and magnitude.** Without normalization, it is possible that two long vectors at a small angle have a larger dot product than two short, perfectly aligned vectors. Therefore, **vectors are often normalized to provide a range from -1 to 1 as a pure measure of directional similarity**:

$$
\begin{align*}
Cosine Similarity = \frac{u \bullet v}{|u||v|} = cos(\theta)
\end{align*}
$$

- Even without normalization, however, the dot product captures similarity through component-wise agreement (large values in the same positions yield a large sum). **Scaling and normalization might refine this measure, but this core principle holds.**

It is extremely important to understand how dot products determine the extent to which each position in a sequence gives importance to any other element. The higher the dot product, the higher the similarity, and the higher the attention score between the two positions.


In [3]:
attn_weights_3_norm = attention_scores_3/attention_scores_3.sum()
print(f'Normalized attention weights for the third token are: {attn_weights_3_norm}')
print(f'The sum of the normalized attention weights for the third token is: {attn_weights_3_norm.sum()}')

Normalized attention weights for the third token are: tensor([0.1452, 0.2875, 0.1757, 0.1497, 0.1815, 0.0605])
The sum of the normalized attention weights for the third token is: 0.9999998807907104


#### Why Normalize?

Above, we have implemented a very straightforward method for achieving normalization. We have normalized the attention scores $w_{3,i}$ to obtain the attention weights $\alpha_{3,i}$. The weights are now more interpretable, and they now sum up to 1. Each attention weight now shows the attention that token $x^{(3)}$ (**the query**) should place on token $x^{(i)}$ (**the key**) in a standardized, proportional manner.

Using the example above (although remember they are meaningless, random embeddings), we see that for the query token $x^{(3)}$, weight $\alpha_{3,2}$ indicates that 28.75% of the attention of $x^{(3)}$ is placed on $x^{(2)}$, while weight $\alpha_{3,6}$ indicates that only 6.05% of the attention is placed on $x^{(6)}$.

In practice, it is more practical and common to use a **softmax function** for normalization, as it is better at managinng extreme values and has better gradient properties for training.

$$
\begin{align*}
softmax(w_{i,j}) = \frac{e^{w_{i,j}}}{\sum_k e^{w_{i,k}}}
\end{align*}
$$

- By converting scores into probabilities between 0 and 1 that sum to 1, normalization prevents weights from growing too large or shrinking too small. Unbounded raw scores could otherwise lead to extreme values, causing gradients to explode or vanish during backpropagation, disrupting convergence.
- The softmax function has a well-behaved, bounded derivative that ensures that gradient updates remain consistent, and that they avoid the instability that might occur with raw scores.

$$
\begin{align*}
\frac{\partial{softmax(x_{j})}}{\partial x_{k}} = softmax(x_{j})(\delta_{j, k} - softmax(x_{k}))
\end{align*}
$$

where

$$
 \begin{equation}
 \delta_{j,k} =
   \left\{\begin{array}{lr}
       1, & i = j \\
       0, & i \mathrel{\mathtt{!=}} j
    \end{array}\right.
 \end{equation}
 $$

In [4]:
def softmax_naive(x):
    return torch.exp(x)/torch.sum(torch.exp(x))

attn_weights_3_norm_softmax = softmax_naive(attention_scores_3)
print(f'Normalized attention weights for the third token after softmax are: {attn_weights_3_norm_softmax}')
print(f'The sum is {attn_weights_3_norm_softmax.sum()}')

#or, using PyTorch softmax:
attn_weights_3_norm_softmax_ = torch.nn.functional.softmax(attention_scores_3, dim=0)
print(f'Normalized attention weights for the third token after softmax are: {attn_weights_3_norm_softmax_}')
print(f'The sum is {attn_weights_3_norm_softmax_.sum()}')


Normalized attention weights for the third token after softmax are: tensor([0.1509, 0.2445, 0.1674, 0.1532, 0.1707, 0.1133])
The sum is 1.0
Normalized attention weights for the third token after softmax are: tensor([0.1509, 0.2445, 0.1674, 0.1532, 0.1707, 0.1133])
The sum is 1.0000001192092896


As a final step, we simply multiply all embedded input tokens $x^{(i)}$ with the corresponding attention weights, and sum everything to obtain context vector $z^{(3)}$.

In [5]:
query = inputs[2]
context_vec_3 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_3 += attn_weights_3_norm_softmax[i] * x_i
print(f'Context vector for the third token is: {context_vec_3}')

Context vector for the third token is: tensor([0.5876, 0.4533, 0.3425])


#### Computing Attention Weights for All Token

We now generalize the code above, which computes and normalizes the attention weights of all keys, $x^{(j)}$ for a given query $x^{(i)}$, to compute the attention weights and context vectors for all inputs.

In [6]:
def calculate_intermediate_attention_score_all_positions(inputs):
    attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])
    for i, x_i in enumerate(inputs):
        for j, x_j in enumerate(inputs):
            attn_scores[i,j] = torch.dot(x_i, x_j)
    return attn_scores


attention_scores = calculate_intermediate_attention_score_all_positions(inputs)
print(f'The intermediate attention scores for all positions are:\n {attention_scores}')

The intermediate attention scores for all positions are:
 tensor([[0.9371, 0.7265, 0.4921, 0.5031, 0.3650, 0.3320],
        [0.7265, 1.6709, 0.9745, 0.9920, 0.9532, 0.3470],
        [0.4921, 0.9745, 0.5957, 0.5074, 0.6152, 0.2050],
        [0.5031, 0.9920, 0.5074, 0.9125, 0.3021, 0.2960],
        [0.3650, 0.9532, 0.6152, 0.3021, 0.7590, 0.1270],
        [0.3320, 0.3470, 0.2050, 0.2960, 0.1270, 0.1400]])


For faster computation, we can avoid using for loops by using matrix multiplication:

In [7]:
attention_scores = inputs @ inputs.T
print(attention_scores)

attention_weights = torch.nn.functional.softmax(attention_scores, dim=1)
print(f'Normalized attention weights for the third token after softmax are: {attention_weights}')

print(f'Row sums: {attention_weights.sum(dim=1)}')

all_context_vectors = attention_weights @ inputs
print(f'Context vectors for all positions are:\n {all_context_vectors}')

tensor([[0.9371, 0.7265, 0.4921, 0.5031, 0.3650, 0.3320],
        [0.7265, 1.6709, 0.9745, 0.9920, 0.9532, 0.3470],
        [0.4921, 0.9745, 0.5957, 0.5074, 0.6152, 0.2050],
        [0.5031, 0.9920, 0.5074, 0.9125, 0.3021, 0.2960],
        [0.3650, 0.9532, 0.6152, 0.3021, 0.7590, 0.1270],
        [0.3320, 0.3470, 0.2050, 0.2960, 0.1270, 0.1400]])
Normalized attention weights for the third token after softmax are: tensor([[0.2376, 0.1925, 0.1522, 0.1539, 0.1341, 0.1297],
        [0.1235, 0.3176, 0.1583, 0.1611, 0.1550, 0.0845],
        [0.1509, 0.2445, 0.1674, 0.1532, 0.1707, 0.1133],
        [0.1477, 0.2408, 0.1483, 0.2224, 0.1208, 0.1201],
        [0.1371, 0.2468, 0.1760, 0.1287, 0.2033, 0.1080],
        [0.1818, 0.1846, 0.1601, 0.1754, 0.1481, 0.1500]])
Row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Context vectors for all positions are:
 tensor([[0.5279, 0.4187, 0.4037],
        [0.6281, 0.5036, 0.3311],
        [0.5876, 0.4533, 0.3425],
        [0.5420, 0.4984, 

### Creating Good Context Vectors with Trainable Weights

Having now implemented a basic, simplified attention mechanism, we will now add trainable weights to the attention mechanism. We will have three trainable matrices of weights: $W_{q}$, $W_{k}$ and $W_{v}$. These will be the ones projecting the embedded input tokens $x_{(i)}$ into **query, key and value vectors**.

- Queries (Q) represent the "questions" or "requests" the model is asking about the input. In self-attention, each input token (e.g., a word in a sentence) generates a query vector to "search" for relevant information.
- Keys (K) act as "labels" or "indices" that the queries compare against to find matches. Each input token also generates a key vector, and, as we will see now, the similarity between a query and a key determines the attention weight.
- Values (V) contain the actual "content" or information that the model retrieves. Once attention weights are computed, they are used to form a weighted combination of value vectors.

To better understand this, imagine a normal database of key-value pairs representing first names (keys) and favorite food (values). Database $\mathcal{D}$ consists of tuples {("Rodrigo", "Tacos"), ("Daniel", "Paella"), ("Diego", "Burger")...("Rodolfo", "Pizza")}. We can operate on $\mathcal{D}$ with the exact query for "Rodrigo", which would return the value "Tacos". If ("Rodrigo", "Tacos") did not exist as a record, then there would be no valid answer. If we allowed approximate matches, then maybe we would retrieve ("Rodolfo", "Pizza") instead. Therefore, if $\mathcal{D}$:

$$
\begin{align*}
\mathcal{D} = {(k_{1}, v_{1}), (k_{2}, v_{2}), ..., (k_{m}, v_{m})}
\end{align*}
$$

denotes a database of $m$ tuples of keys and values, and $q$ denotes a query against this database:

- We can design $q$ that operates on $(k,v)$ pairs to be valid, no matter the database size or the existence of perfect matches.
- The same $q$ can receive different answers, depending on the contents of the database, and on the way approximate matches are computed.
- There is no need to compress the database to make the operations effective.


> Note: Do not confuse attention weights with weight parameters. As we have already learned, attention weights determine the extent to which a context vector depends on the different parts of the input. Weight parameters are learned, and they define the behavior of the network's connections. Attention weights are dynamic and context-dependent.
>
> The matrices transforming the input into Q, K and V, the weight parameters, are learned during training. This means that a model can fine-tune how it "asks questions" (queries), "indexes information" (keys), and "represents content" (values) based on the task (e.g., translation, classification, etc).

At each step, we compute query $(q)$, key $(k)$ and value $(v)$ vectors via matrix multiplication between the input $x^{(i)}$ and the weight matrix $W_{q}$ (for the query vector), $W_{k}$ (for the key vector) and $W_{v}$ (for the value vector).

|           ![Self-Attention Mechanism Step](../images/kqv.png)           |
|:--------------------------------------------------------------------:|
| *Understanding Keys, Queries & Values* (Akshay Pachaar, on LinkedIn) |

Let's compute the context vector $z^{(3)}$ first, and then extend the logic to compute all other context vectors.

In [8]:
x_3 = inputs[2]
d_in = inputs.shape[1] # input embedding size
d_out = 2 #output embedding size

torch.manual_seed(123)
W_q = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False) # in real training scenarios, these would be set to requires_grad=True
W_k = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False) # for now it is just to avoid varying outputs
W_v = torch.nn.Parameter(torch.randn(d_in, d_out), requires_grad=False)

query_3_first = x_3 @ W_q
key_3_first = x_3 @ W_k
value_3_first = x_3 @ W_v

print(query_3_first)

tensor([-0.4854,  0.0467])


In [9]:
print(f'Inputs have shape: {inputs.shape}')

keys = inputs @ W_k
values = inputs @ W_v

print(f'Keys has shape: {keys.shape}')
print(f'Values has shape: {values.shape}')

Inputs have shape: torch.Size([6, 3])
Keys has shape: torch.Size([6, 2])
Values has shape: torch.Size([6, 2])


The next step is to compute an unscaled attention score $\omega_{3,i}$ for all keys $k^{(i)}$ with respect to query $q^{(3)}$ (since we are trying to compute the context vector for the third token). The attention score is simply the dot product between the query and the key vectors. Computing the attention score $w_{3,3}$, for instance:

In [10]:
keys_3 = keys[2]
attn_score_33 = query_3_first.dot(keys_3)
print(f'Attention score for w_33:\n {attn_score_33}')

# generalizing to all attention scores for query_3
attn_scores_3 = query_3_first @ keys.T
print(f'Attention scores for w_3i:\n {attn_scores_3}')
print(f'Shape of attention scores for w_31:\n {attn_scores_3.shape}')

Attention score for w_33:
 0.19979067146778107
Attention scores for w_3i:
 tensor([ 0.0214,  0.2577,  0.1998, -0.0948,  0.3484, -0.0249])
Shape of attention scores for w_31:
 torch.Size([6])


|                                                                                     ![Query Intuition](../images/querylogic.png)                                                                                      |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| *Query $q^{(3)}$ ($q^{(2)}$ in this example) is used to calculate all attention scores $w_{3,i}$* (Understanding and Coding the Self-Attention Mechanism of Large Language Models from Scratch, Sebastian Raschka) |

As before, we now compute the attention weights by normalizing the attention scores through a softmax function. This time, we scale the attention scores by dividing them by the square root of the embedding dimension of the keys. The **scaled-dot product attention weights** can be written as:

$$
\alpha_{i,j} = \text{softmax}\left(\frac{\mathbf{q}_i \bullet \mathbf{k}_j}{\sqrt{d_k}}\right)
$$

where

$$
\begin{align*}
\text{softmax}(x_j) = \frac{e^{x_j}}{\sum_k e^{x_k}}
\end{align*}
$$

> The scaling factor $\frac{1}{\sqrt{d_k}}$ is very important. Without it, dot product grows with the dimension $d_k$, leading to potentially very large values, especially when scaling up the embedding dimension (typically greater than 1000). For example, if $d_k$ = 64, the dot product's magnitude might be around 8 (since variance scales with $d_k$). Large scores push the softmax function into a 'saturated' regime, where one weight approaches 1 while others approach 0, resulting in very tiny gradients for most tokens, which can slow down or completely stagnate learning.
>
> Without scaling, scores might be [8,0,0]:
$$
\begin{align*}
\text{softmax}([8,0,0]) \approx [1,0,0]
\end{align*}
$$

>All gradients but one are nearly 0, stalling learning. With scaling $\frac{8}{\sqrt{64}} = 1$:

$$
\begin{align*}
\text{softmax}([1,0,0]) \approx [0.576, 0.211, 0.211]
\end{align*}
$$
>Now, all tokens receive non-zero gradients.



In [11]:
d_k = keys.shape[-1]
print(f'Embedding dimension of keys is: {d_k}')
attn_weights_3 = torch.nn.functional.softmax(attn_scores_3/d_k**0.5, dim=0)
print(f'Attention weights for the third token are: {attn_weights_3}')


Embedding dimension of keys is: 2
Attention weights for the third token are: tensor([0.1547, 0.1828, 0.1755, 0.1425, 0.1949, 0.1497])


As a last step, we multiply all value vectors $v^{(i)}$ with their respective attention weights $\alpha_{3,i}$, and then sum everything to obtain the context vector $z^{(3)}$.

In [12]:
context_vec_3 = attn_weights_3 @ values
print(f'Context vector for the third token is: {context_vec_3}')

Context vector for the third token is: tensor([0.2618, 0.4683])


#### Putting it All Together

In [13]:
class SelfAttentionBasic(torch.nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_q = torch.nn.Parameter(torch.randn(d_in, d_out))
        self.W_k = torch.nn.Parameter(torch.randn(d_in, d_out))
        self.W_v = torch.nn.Parameter(torch.randn(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_k
        queries = x @ self.W_q
        values = x @ self.W_v
        scores = queries @ keys.T
        attn_weights = torch.nn.functional.softmax(scores/keys.shape[-1]**0.5, dim=1)
        context = attn_weights @ values
        return context

torch.manual_seed(123)
selfattentionv1 = SelfAttentionBasic(d_in, d_out)
context_vec_3_v1 = selfattentionv1(inputs)
print(f'Context vector for the third token using v1 is:\n {context_vec_3_v1}')

class SelfAttentionBasicv2(torch.nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_q = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)
        scores = queries @ keys.T
        attn_weights = torch.nn.functional.softmax(scores/keys.shape[-1]**0.5, dim=1)
        context = attn_weights @ values
        return context

torch.manual_seed(123)
selfattentionv2 = SelfAttentionBasicv2(d_in, d_out)
context_vec_3_v2 = selfattentionv2(inputs)
print(f'Context vector for the third token using v2 is:\n {context_vec_3_v2}')

Context vector for the third token using v1 is:
 tensor([[0.2695, 0.5081],
        [0.2669, 0.4900],
        [0.2618, 0.4683],
        [0.2682, 0.4938],
        [0.2571, 0.4477],
        [0.2615, 0.4663]], grad_fn=<MmBackward0>)
Context vector for the third token using v2 is:
 tensor([[-0.4850, -0.1109],
        [-0.4830, -0.1133],
        [-0.4813, -0.1111],
        [-0.4783, -0.1119],
        [-0.4815, -0.1107],
        [-0.4786, -0.1096]], grad_fn=<MmBackward0>)


In the code block above, there are two classes with the same goal: `SelfAttentionBasic` (v1) and `SelfAttentionBasicv2` (v2). Note that, in v1, the weight matrices for queries $W_q$, keys $W_k$ and values $W_v$ are defined using `torch.nn.Parameter` objects, where each weight matrix is initialized as a tensor of shape (`d_in`, `d_out`). Therefore, the matrix multiplication results in queries with shape `(num_tokens, d_out)`.

In [14]:
print(selfattentionv1.W_q.shape) #d_in, d_out

torch.Size([3, 2])


However, in v2, the weight matrices are implemented using `torch.nn.Linear` layers, a module that internally stores a weight matrix, accessible as `self.W_q.weight`, with shape (`d_out, d_in`). Therefore its transpose `self.W_q.weight.T` is (`d_in, d_out`), matching the output shape in v1.

In [15]:
print(selfattentionv2.W_q.weight.shape)

torch.Size([2, 3])


In [16]:
v1_weights = selfattentionv1.state_dict()
v2_weights = selfattentionv2.state_dict()
print(f'Weights for v1:\n {v1_weights}')
print(f'Weights for v2:\n {v2_weights}')

Weights for v1:
 OrderedDict([('W_q', tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]])), ('W_k', tensor([[-0.9724, -0.7550],
        [ 0.3239, -0.1085],
        [ 0.2103, -0.3908]])), ('W_v', tensor([[ 0.2350,  0.6653],
        [ 0.3528,  0.9728],
        [-0.0386, -0.8861]]))])
Weights for v2:
 OrderedDict([('W_q.weight', tensor([[-0.2354,  0.0191, -0.2867],
        [ 0.2177, -0.4919,  0.4232]])), ('W_k.weight', tensor([[-0.4196, -0.4590, -0.3648],
        [ 0.2615, -0.2133,  0.2161]])), ('W_v.weight', tensor([[-0.4900, -0.3503, -0.2120],
        [-0.1135, -0.4404,  0.3780]]))])


> While understanding the various existing weight storing conventions seems like a minor technical detail, it has very important practical implications. Suppose that we wanted to match a known really good set from v1, and that we want to initialize the weights in v2 with this known set of weights. If we naively set `self.W_q.weight.data = W.q`, we would get a shape mismatch. Our model would fail due to incompatible dimensions, and misunderstanding this transpose would throw our comparison, and our debugging, completely off.

In [17]:
selfattentionv2.W_v.weight.data = W_v.T
selfattentionv2.W_k.weight.data = W_k.T
selfattentionv2.W_q.weight.data = W_q.T

context_vec_3_v2_t = selfattentionv2(inputs)
print(f'Context vector for the third token is:\n {context_vec_3_v2_t}')



Context vector for the third token is:
 tensor([[0.2695, 0.5081],
        [0.2669, 0.4900],
        [0.2618, 0.4683],
        [0.2682, 0.4938],
        [0.2571, 0.4477],
        [0.2615, 0.4663]], grad_fn=<MmBackward0>)


**Now the resulting context vectors are the same!!!**

|                              ![Complete Self-Attention](../images/completeselfattention.png)                              |
|:----------------------------------------------------------------------------------------------------------------------:|
| Self-Attention Forward Pass Visualized (Understanding the Self-Attention Mechanism (Damien Benveniste on *The AiEdge*) |

### Causal Attention
In a standard left-to-right language model, we predict the next token $x_t$ given all previous tokens ($x_1, x_2,..., x_{t-1}$). With our current implementation of Self-Attention, our model could "see" $x_{t+1}, x_{t+2}, ...$ and could potentially trivially learn to cheat by reading the true next words instead of modeling a distribution $p(x_{t}|x_{<t})$. Ideally, we want each hidden state $h_t$ (and therefore each output distribution) to depend *only* on past and present inputs. This would enforce a clean factorization of the joint probability:

$$
\begin{align*}
p(x_1, x_2,\dots, x_T) = \prod_{t=1}^{T} p(x_t | x_1,\dots, x_{t-1})
\end{align*}
$$

**Causal attention** is a form of self-attention that restricts a model to only consider previous and current inputs in a sequence when processing a token and computing attention scores. Language models, and any problem where we must generate or predict in sequence (time-series forecasting, music generation, control signs in robotics) use the same principle: **do not peek into the future.**

|                      ![Causal Attention](../images/causalattention.png)                       |
|:------------------------------------------------------------------------------------------:|
| Causal Self-Attention (Aisuko on Github *Kaggle Notebook Causal Self-Attention Version 6*) |

As can be seen in the illustration above, the way causal attention is achieved is by masking out the attention weights above the diagonal, in such a way that token $x^{(i)}$ only has access to attention weights corresponding to tokens $x^{(<i)}$. Formally:

Recall:

$$
\begin{align*}
\text{Attention}(Q,K,V) = \text{softmax}(\frac{QK^T}{\sqrt{d_k}})V
\end{align*}
$$

To enforce the "no peeking forward" constrain, we add a mask matrix $M \in R^{TxT}$ before the softmax:

$$
\begin{align*}
A = \frac{QK^T}{\sqrt{d_k}} + M
\end{align*}
$$

where

$$
 \begin{equation}
 M_{i,j} =
   \left\{\begin{array}{lr}
       0, & j <= i \\
       -\infty, & j > 1
    \end{array}\right.
 \end{equation}
 $$

Then,

$$
\begin{align*}
\text{CausalAttention}(Q,K,V) = \text{softmax}(A)V,
\end{align*}
$$

so that for each query at position $i$, all keys $j>i$ get weight zero.





In [18]:
queries = selfattentionv2.W_q(inputs)
keys = selfattentionv2.W_k(inputs)
scores = queries @ keys.T
weights = torch.nn.functional.softmax(scores/keys.shape[-1]**0.5, dim=1)
print(weights)

context_len = scores.shape[0]
causal_mask = torch.tril(torch.ones(context_len, context_len))

tensor([[0.1364, 0.1998, 0.1845, 0.1137, 0.2363, 0.1292],
        [0.1475, 0.1955, 0.1798, 0.1277, 0.2129, 0.1366],
        [0.1547, 0.1828, 0.1755, 0.1425, 0.1949, 0.1497],
        [0.1480, 0.1996, 0.1798, 0.1261, 0.2132, 0.1333],
        [0.1615, 0.1708, 0.1703, 0.1577, 0.1775, 0.1622],
        [0.1558, 0.1822, 0.1748, 0.1441, 0.1926, 0.1505]],
       grad_fn=<SoftmaxBackward0>)


> We can implement the mask using PyTorch's `tril` function, which creates a given mask where the values above the diagonal are zero.

In [19]:
add_mask = (causal_mask == 0).float() * -torch.inf
add_mask = torch.nan_to_num(add_mask, nan=0.0)
print(f'The mask looks like this:\n {add_mask}')

The mask looks like this:
 tensor([[ 0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -3.4028e+38,
         -3.4028e+38],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -3.4028e+38],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00]])


Note that we apply the mask **before applying the softmax normalization**. The softmax function converts its inputs into a probability distribution, and when negative infinity values are present in a row, the softmax function treats them as zero probability ($e^{-\infty} \approx 0$). This is an efficient masking trick!!.

In [20]:
masked = scores + add_mask
print(f'Masked scores:\n {masked}')

attn_weights = torch.nn.functional.softmax(masked/keys.shape[-1]**0.5, dim=1)

print(f'Attention weights with causal mask:\n {attn_weights}')

Masked scores:
 tensor([[ 3.9294e-04, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 8.7802e-02,  4.8573e-01, -3.4028e+38, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 2.1387e-02,  2.5766e-01,  1.9979e-01, -3.4028e+38, -3.4028e+38,
         -3.4028e+38],
        [ 1.5009e-01,  5.7294e-01,  4.2477e-01, -7.6155e-02, -3.4028e+38,
         -3.4028e+38],
        [-3.0680e-02,  4.9062e-02,  4.4648e-02, -6.3659e-02,  1.0317e-01,
         -3.4028e+38],
        [ 2.9634e-02,  2.5077e-01,  1.9277e-01, -8.0661e-02,  3.2978e-01,
         -1.9344e-02]], grad_fn=<AddBackward0>)
Attention weights with causal mask:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4301, 0.5699, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3015, 0.3564, 0.3421, 0.0000, 0.0000, 0.0000],
        [0.2265, 0.3054, 0.2751, 0.1930, 0.0000, 0.0000],
        [0.1927, 0.2039, 0.2033, 0.1883, 0.2118, 0.0000],
        [0.1558, 0.1822, 0.1748, 0.1441, 0.1926, 0

#### Causal Attention with Dropout

|      ![Dropout in Causal Attention](../images/dropout.png)       |
|:-------------------------------------------------------------:|
| Build a Large Language Model from Scratch (Sebastian Raschka) |

In deep models, attention heads can "collapse" onto a few dominant key-query pairs and learn spurious correlations. **Dropout** randomly silences some of these weights **during training**, forcing the model to distribute its focus more broadly, and preventing over-reliance on specific patterns. This idea is even more relevant when training on limited text, where overfitting is an even bigger risk. With dropout, at inference, the full ensemble of "thinned" sub-networks yields a smoother, more stable prediction. Formally:

$$
\begin{align*}
A = \frac{QK^T}{\sqrt{d_k}} + M, \alpha = \text{softmax}(A)
\end{align*}
$$

we introduce a dropout mask $D \in {0,1}^{TxT}$ sampled as:

$$
\begin{align*}
D_{i,j} &\sim \text{Bernoulli}(p)
\end{align*}
$$

where $p$ is the keep probability (1-dropout_rate). We can then define the dropped-out attention weights:

$$
\begin{align*}
\widetilde{\alpha}_{i,j} = \frac{D_{i,j}}{P}\alpha_{i,j}
\end{align*}
$$

Note that multiplying by $frac{1}{p}$ ensures that the expected outpute remains unchanged:

$$
\begin{align*}
\mathbb{E}[\widetilde{\alpha}_{i,j}] = \alpha_{i,j}, \quad
\text{CausalDropoutAttention}(Q, K, V) = \widetilde{\alpha} V
\end{align*}
$$

since

$$
\mathbb{E}[\frac{D_{i,j}}{p}\alpha_{i,j}] = \frac{1}{p}\alpha_{i,j}\mathbb{E}[D_{i,j}] = \frac{1}{p}\alpha_{i,j}p = \alpha_{i,j}
$$

The $\frac{1}{p}$ factor therefore preserves the overall magnitude of the attention-weighted sum. **Without this scaling, the expected sum would diminish by a factor $p$, forcing us to re-tune other parameters.** Keeping the expected activations constant means optimizer hyperparamaters do not have to compensate for a shifting scale as dropout rate changes.


> Dropout is typically applied at two stages: after calculating attention weights, or after applying the attention weights to the value vectors. In practice, the more common variant is applying after calculating attention weights.




In [21]:
print(f'Attention weights after dropout:\n {torch.nn.Dropout(p=0.5)(attn_weights)}')

Attention weights after dropout:
 tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8602, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6031, 0.7127, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6109, 0.5501, 0.3860, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3116, 0.3643, 0.0000, 0.0000, 0.3852, 0.3010]],
       grad_fn=<MulBackward0>)


In [22]:
class CausalAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout_rate=0.1, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        # linear projections for queries, keys and values
        self.W_q = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        # dropout layer for attention weights
        self.dropout = torch.nn.Dropout(p=dropout_rate)
        # causal mask to prevent attending to future tokens
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, inputs: torch.Tensor):
        b, seq_len, d_in = inputs.shape # shape: (batch_size, seq_len, d_in)

        keys = self.W_k(inputs) # shape: (batch_size, seq_len, d_in)
        queries = self.W_q(inputs)
        values = self.W_v(inputs)

        attn_scores = queries @ keys.transpose(-2, -1) # shape: (batch_size, seq_len, seq_len)
        mask = self.mask[:seq_len, :seq_len].bool() #slice mask to match sequence length
        attn_scores.masked_fill_(mask, -torch.inf)

        attn_weights = self.dropout(torch.nn.functional.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1))
        context = attn_weights @ values
        return context

In [23]:
batch = torch.stack((inputs, inputs), dim = 0) #simulate batch inputs
torch.manual_seed(123)
context_len = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_len, 0.0)
context_vecs = ca(batch)
print(f'Context vectors shape:\n {context_vecs.shape}')

Context vectors shape:
 torch.Size([2, 6, 2])


### From Single-Head Attention to Multi-Head Attention

Multi-head attention methods divide the attention mechanism into multipe "heads". Each head operates independently, and one head is a causal attention mechanism. Therefore, a single causal attention mechanism, like the one we have developed thus far, would be considered a "single-head attention"

Recall that transformer architectures were originally proposed as a more efficient seq2seq model which improved the common RNN encoder-decoder architectures, which processed tokens one at a time and carried hidden state vectors that recursively accumulated information. An RNN can therefore implement very high-capacity, sequential transformations, where each new hidden state is a learned function of the previous hidden state and the current input. Transformers replaced recurrence with self-attention:

>'In these (RNN) models, the number of operations required to relate signals from two arbitrary input or output positions grows in the distance between positions [...]. This makes it more difficult to learn dependencies between distant positions. In the Transformer this is reduced to a constant number of operations...
>
>--<cite>Attention is All You Need, Vaswani, Shazeer, Parmar, Uszkoreit, Jones, Gomez, Kaiser, Polosukhin, 2017</cite>

All tokens now directly attend all others in a single shot. This brings huge efficiency gains (parallelism, constant "path length" between distant tokens) although also some drawbacks:

> 'In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect that we counteract with Multi-Head Attention...
>
>-- <cite>Attention is All You Need, Vaswani et al., 2017</cite>

At each layer, we are computing a weight matrix $\alpha$, and multiplying this single distribution against the entire value matrix $V$. In effect, we are averaging all token-representaitons according to a single pattern, per query. *However, it is perfectly possible that there are multiple patterns that are useful for different queries, and that a single vector of attention weights cannot simultaneously capture all of them.* In other words, a single vector of attention weights cannot, for example, capture both subject-verb agreement and long-range topic coherence in one go. It's ability to do that is inhibited by averaging (while RNNs are more expressive in that they mix dozens of past tokens at each time step, stacking $T$ nonlinear layers and being unlimited in their capacity to rearrange and re-weight all past information in arbitrarily complex ways).

To recover and exceed that lost expressiveness without reintroducing recurrence, Transformers use $H$ parallel attention heads. The input embedding $X \in R^{Txd}$ is split into $H$ heads, each of which computes a different attention pattern. That is, from X we compute $H$ different query, key and value matrices which are then linearly projected with different and independent linear projections to $d_k, d_k, d_v$ dimensions, respectively. Then, on each one of these dimensions, we perform the attention mechanism in parallel. The heads are then concatenated and projected back into a single vector. Formally:

$$
\begin{align*}
\text{MultiHeadAttention}(Q,K,V) = \text{Concat}(\text{head}_1,\dots, \text{head}_h)W^O
\end{align*}
$$

where

$$
\begin{align*}
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
\end{align*}
$$

where $W_i^Q \in R^{d_{model} \times d_k}$, $W_i^K \in R^{d_{model} \times d_k}$, $W_i^V \in R^{d_{model} \times d_v}$ and $W^O \in R^{hd_v \times d_{model}}$, the projections, are trainable weight (parameters), and where $d_k = d_v = d_{model}/H$ is the per-head dimensionality.

**Multi-head attention hence allows the model to learn multiple feature mappings, making them "free" to learn different functions and consider, for each input, different metrics of what is "relevant".** When we combine them, the hope is to get a richer, more nuanced representation than if one big attention were trying to do everything.

> Just as a convolutional layer uses multiple filters to detect different underlying patterns (e.g. edge detectors, texture detectors, etc) on the same image, where each new filter complements the previous ones, multi-head attention uses multiple attention heads to detect different patterns in the same input sequence. Each head is free to learn a different notion of relevance, and the final layer learns to fuse them.

|              ![Multi-Head Attention](../images/multihead.png)               |
|:------------------------------------------------------------------------:|
| Multi-Head Attention (*Attention is All You Need, Vaswani et. al, 2017*) |

> It might be important to note that, even if we are running attention $H$ times in parallel, we do not blow up our compute budget, because **each head works in a smaller subspace**. In fact, we can show that the overall cost of H heads of size $d_{model}/H$ is the same as a single head of size $d_model$:
> For a single head, we must compute:
$$
Q, K, V = XW_Q, XW_K, XW_V
$$
> where $W_Q, W_K, W_V \in R^{d_{model} \times d_{k}}$ and $X \in R^{T \times d_{model}}$. This requires $T \times d_{model} \times d_k$ operations, where $T$ is the sequence length (num. of tokens).
> Therefore, the cost of projection is $(T \times d_{model} \times d_k)$ and since we have three projections for each head, $(3T \times d_{model} \times d_k)$.
> We then compute the scaled dot-product attention, which requires $(T^2 \times d_k)$ operations, and multiplying by V requires $(T^2 \times d_k)$ operations. Therefore, the total cost of a single head is $(3T \times d_{model} \times d_k + 2T^2 \times d_k) \approx O(T^2 \times d_k + T \times d_{model} \times d_k)$
>Now, for Multi-Head attention, we run $H$ heads in parallel, each of size $d_k = d_{model}/H$:
$$
\begin{align*}
H \times (3T \times d_{model} \times d_k + T^2 \times d_k) \\
= (3T \times d_{model} \times (H \times d_k) + T^2 \times (H \times d_k)) \\
= (3T \times d_{model} \times d_k + T^2 \times d_k) \\
\approx O(T^2 \times d_k + T \times d_{model} \times d_k)
\end{align*}
$$
> **Multi-Head attention has the same order a single attention of full size $d_{model}$**. **By choosing each head to live in a reduced dimension and then concatenating them, we preserve overall dimensionality**


In [24]:
class MultiHeadAttentionSequential(torch.nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length, dropout_rate=0.1, qkv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout_rate, qkv_bias) for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

torch.manual_seed(123)
context_length = batch.shape[1] # num of words in batch input
d_in, d_out = 3, 2
mha = MultiHeadAttentionSequential(d_in, d_out, num_heads=2, dropout_rate=0.0, context_length=context_length)
context_vecs = mha(batch)

print(f'The context vectors using Multi-head attention are:\n {context_vecs}')
print(f'The shape of the context vectors is:\n {context_vecs.shape}')


The context vectors using Multi-head attention are:
 tensor([[[-0.4127,  0.2307,  0.4378,  0.0873],
         [-0.6237, -0.0703,  0.6273,  0.3715],
         [-0.5776, -0.0877,  0.5705,  0.3396],
         [-0.5508, -0.1369,  0.5347,  0.3698],
         [-0.5376, -0.1341,  0.5190,  0.3398],
         [-0.4786, -0.1096,  0.4648,  0.3034]],

        [[-0.4127,  0.2307,  0.4378,  0.0873],
         [-0.6237, -0.0703,  0.6273,  0.3715],
         [-0.5776, -0.0877,  0.5705,  0.3396],
         [-0.5508, -0.1369,  0.5347,  0.3698],
         [-0.5376, -0.1341,  0.5190,  0.3398],
         [-0.4786, -0.1096,  0.4648,  0.3034]]], grad_fn=<CatBackward0>)
The shape of the context vectors is:
 torch.Size([2, 6, 4])


The first dimension of the output is the number of batches processed, the second dimension is the number of tokens in the input sequence, and the third dimension is the number of heads times the output dimension of each head. Since `d_in = 3` and `d_out = 2`, the output dimension of each head is 2, and since we have 2 heads, the output dimension of the MultiHeadAttention module is `num_heads * d_out` = 4. Since the input texts are duplicated:
```python
batch = torch.stack((inputs, inputs), dim = 0)
```
The context vectors are the same for both batches.

If, for instance, we wanted to keep `num_heads = 2` and at the same time have the output context vectors have a dimension of 2, we could set `d_out = 1`:

In [25]:
d_in, d_out = 3, 1
mha = MultiHeadAttentionSequential(d_in, d_out, num_heads=2, dropout_rate=0.0, context_length=context_length)
context_vecs = mha(batch)

print(f'The context vectors using Multi-head attention are:\n {context_vecs}')
print(f'The shape of the context vectors is:\n {context_vecs.shape}')

The context vectors using Multi-head attention are:
 tensor([[[0.0231, 0.2628],
         [0.1894, 0.2801],
         [0.1759, 0.2473],
         [0.2453, 0.2401],
         [0.2036, 0.2196],
         [0.1842, 0.2009]],

        [[0.0231, 0.2628],
         [0.1894, 0.2801],
         [0.1759, 0.2473],
         [0.2453, 0.2401],
         [0.2036, 0.2196],
         [0.1842, 0.2009]]], grad_fn=<CatBackward0>)
The shape of the context vectors is:
 torch.Size([2, 6, 2])


`MultiHeadAttentionSequential` processes each individual head sequentially, employing a for loop for a list of `CausalAttention` objects. A faster, more optimal implementation of Multi-Head Attention is presented here:

In [26]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length, dropout_rate=0.1, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0) # d_out must be divisible by num_heads
        self.d_out = d_out
        self.num_heads = num_heads
        self.d_head = d_out // num_heads # dimension of each head

        #separate linear projections for queries, keys and values
        self.W_q = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_k = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_v = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = torch.nn.Dropout(p=dropout_rate)

        # causal mask, ones above the diagonal
        mask = torch.triu(torch.ones(context_length, context_length), diagonal=1).bool()
        self.register_buffer('mask', mask)

        self.out_proj = torch.nn.Linear(d_out, d_out) # a linear layer to combine all head outputs
        self.scale = self.d_head ** -0.5

    def forward(self, x):
        """
        x: (batch_size, seq_len, d_in)
        returns: (batch_size, seq_len, d_out)
        """
        b, num_tokens, d_in = x.shape
        keys = self.W_k(x) # shape: (batch_size, seq_len, d_in)
        queries = self.W_q(x)
        values = self.W_v(x)

        # split the keys, queries and values into num_heads heads.
        def split_heads(tensor):
            """
            split Q,K,V into heads and permute to (batch_size, num_heads, seq_len, d_in)
            """
            return (
                tensor
                .reshape(b, num_tokens, self.num_heads, self.d_head)
                .permute(0, 2, 1, 3)
            )

        queries, keys, values = split_heads(queries), split_heads(keys), split_heads(values)


        #the transpose transposes from shape (batch_size, num_heads, num_tokens, d_head) to (batch_size, num_tokens, num_heads, d_head)
        attn_scores = queries @ keys.transpose(-2, -1) #--> (batch_size, num_heads, num_tokens, d_head)
        mask = self.mask[:num_tokens, :num_tokens] #slice mask to match sequence length
        attn_scores_masked = attn_scores.masked_fill(mask, -torch.inf)

        #softmax and dropout
        attn_weights = self.dropout(torch.nn.functional.softmax(attn_scores_masked * self.scale, dim=-1))
        attn_weights = self.dropout(attn_weights)

        context = attn_weights @ values # shape: (batch_size, num_heads, num_tokens, d_head)
        context = context.permute(0, 2, 1, 3).reshape(b, num_tokens, self.d_out)
        context = self.out_proj(context)
        return context

In [29]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length=context_length, dropout_rate=0.0, num_heads=2)
context_vecs = mha(batch)
print(f'The context vectors using Multi-head attention are:\n {context_vecs}')
print(f'The shape of the context vectors is:\n {context_vecs.shape}')

The context vectors using Multi-head attention are:
 tensor([[[0.3145, 0.5066],
         [0.2842, 0.3614],
         [0.2712, 0.3817],
         [0.2544, 0.3887],
         [0.2535, 0.3958],
         [0.2488, 0.4286]],

        [[0.3145, 0.5066],
         [0.2842, 0.3614],
         [0.2712, 0.3817],
         [0.2544, 0.3887],
         [0.2535, 0.3958],
         [0.2488, 0.4286]]], grad_fn=<AddBackward0>)
The shape of the context vectors is:
 torch.Size([2, 6, 2])


> The helper `split_heads` reshapes `(batch_size, num_tokens, d_out)` into `(batch_size, num_words, num_heads, d_head)`, then permutes to `(batch_size, num_heads, num_tokens, d_head)` so that each head can attend in parallel (We are aliging the dimensions to enable dot products).
> After computing `context` for each head, we permute back to `(batch_size, num_tokens, num_heads, d_head)` and reshape to `(batch_size, num_tokens, d_out)`. **Note that, this time, `d_out` dictates the output dimension**.

### That is the end of our exploration of the attention mechanism. We now move on to the architecture of Large Language Models. See `stage1/architecture.ipynb`.